## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users/ritwi/OneDrive/Desktop/Classwork/Weekly Challenges/Challenge 6/Weather_Data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Ayame,5.6083,-3.1563,76.59,95,16,2.84,CI,few clouds
1,1,Sorong,-0.8833,131.2500,82.56,77,95,0.69,ID,overcast clouds
2,2,Hermanus,-34.4187,19.2345,57.00,95,63,1.01,ZA,broken clouds
3,3,Albany,42.6001,-73.9662,53.60,40,20,6.91,US,few clouds
4,4,Yomou,7.5603,-9.2653,72.07,94,100,2.73,GN,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 55.50
What is the maximum temperature you would like for your trip? 65.50


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
2,2,Hermanus,-34.4187,19.2345,57.00,95,63,1.01,ZA,broken clouds
8,8,Bluff,-46.6000,168.3333,59.00,78,96,11.99,NZ,overcast clouds
9,9,Cape Town,-33.9258,18.4232,62.01,94,0,5.75,ZA,clear sky
11,11,Jamestown,42.0970,-79.2353,62.60,51,1,5.75,US,clear sky
12,12,Macia,-25.0269,33.0989,65.46,82,44,2.57,MZ,scattered clouds
19,19,Hasaki,35.7333,140.8333,60.01,54,20,8.05,JP,few clouds
24,24,Kaitangata,-46.2817,169.8464,59.29,62,100,3.02,NZ,overcast clouds
30,30,Gobabis,-22.4500,18.9667,65.05,60,9,8.52,NaN,clear sky
32,32,East London,-33.0153,27.9116,65.01,84,67,11.88,ZA,broken clouds
51,51,Port Alfred,-33.5906,26.8910,62.42,85,54,8.72,ZA,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                4
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                79
City                   79
Lat                    79
Lng                    79
Max Temp               79
Humidity               79
Cloudiness             79
Wind Speed             79
Country                79
Current Description    79
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hermanus,ZA,57.00,broken clouds,-34.4187,19.2345,
8,Bluff,NZ,59.00,overcast clouds,-46.6000,168.3333,
9,Cape Town,ZA,62.01,clear sky,-33.9258,18.4232,
11,Jamestown,US,62.60,clear sky,42.0970,-79.2353,
12,Macia,MZ,65.46,scattered clouds,-25.0269,33.0989,
19,Hasaki,JP,60.01,few clouds,35.7333,140.8333,
24,Kaitangata,NZ,59.29,overcast clouds,-46.2817,169.8464,
32,East London,ZA,65.01,broken clouds,-33.0153,27.9116,
51,Port Alfred,ZA,62.42,broken clouds,-33.5906,26.8910,
61,Buraydah,SA,64.40,clear sky,26.3260,43.9750,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": " , "
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
        lat = row["Lat"]
        lng = row["Lng"]
        params["location"] = f"{lat},{lng}" 
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 
    

    # 6e. Make request and retrieve the JSON data from the search.
        hotels = requests.get(base_url, params=params).json()
    

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        try:  
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        except (IndexError):
            print("Hotel not found... skipping.")          

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hermanus,ZA,57.00,broken clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
8,Bluff,NZ,59.00,overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
9,Cape Town,ZA,62.01,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
11,Jamestown,US,62.60,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
12,Macia,MZ,65.46,scattered clouds,-25.0269,33.0989,Tuck shop my life dcoza
...,...,...,...,...,...,...,...
679,Camacha,PT,62.60,broken clouds,33.0833,-16.3333,Hotel Porto Santo & Spa
687,Mugumu,TZ,58.06,broken clouds,-1.8500,34.7000,Lemala Mara
689,Dilla,ET,60.62,broken clouds,6.4100,38.3100,Afomiya Pension
693,Mwinilunga,ZM,64.42,overcast clouds,-11.7358,24.4293,Kwakuwahi Lodge


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "C:/Users/ritwi/OneDrive/Desktop/Classwork/Weekly Challenges/Challenge 6/Weather_Data/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

In [14]:
fig

Figure(layout=FigureLayout(height='420px'))